In [1]:
!pip install -r requirements.txt

In [2]:
import chainlit as cl

from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.storage import LocalFileStore
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from utils import ArxivLoader, PineconeIndexer

2023-12-14 14:04:46 - Loaded .env file


/Users/mouadhadji/anaconda3/envs/llmops-env/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
system_template = """
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".

Context:
{context}
"""

In [4]:
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

prompt = ChatPromptTemplate(messages=messages)
chain_type_kwargs = {"prompt": prompt}

In [5]:
# load documents from Arxiv
axloader = ArxivLoader()
axloader.main()

2023-12-14 14:04:52 - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=Nuclear+Fission&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=100
2023-12-14 14:04:54 - Got first page: 100 of 100362 total results


In [6]:
len(axloader.documents)

5

In [9]:
import pinecone

In [16]:
try: 
    pinecone.delete_index('arxiv-paper-index')
except:
    pass

In [17]:
# build index in Pinecone
pi = PineconeIndexer()
pi.load_embedder()

In [18]:
pi.index_documents(axloader.documents)

  0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
pinecone.describe_index('arxiv-paper-index')

IndexDescription(name='arxiv-paper-index', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [19]:
llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0
    )

In [20]:
chain = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff",
        return_source_documents=True,
        retriever=pi.get_vectorstore().as_retriever(),
        chain_type_kwargs = {"prompt": prompt}
    )

/Users/mouadhadji/anaconda3/envs/llmops-env/lib/python3.11/site-packages/langchain_community/vectorstores/pinecone.py:62: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [21]:
retriever=pi.get_vectorstore().as_retriever()
retriever.invoke("What is nuclear fission?")

/Users/mouadhadji/anaconda3/envs/llmops-env/lib/python3.11/site-packages/langchain_community/vectorstores/pinecone.py:62: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


[Document(page_content='89, 014323 (2014).\n[10] H.J. KrappeandK.Pomorski, TheoryofNuclear Fission\n(Springer-Verlag, Berlin 2012).\n[11] J.S. Langer, Ann. Phys. (N.Y.) 41, 108(1967).\n[12] I. Aﬄeck, Phys. Rev. Lett 46, 388 (1981).\n[13] K. Hagino, N. Takigawa, and M. Abe, Phys. Rev. C 53,\n1840 (1996).\n[14] M. Kryvohuz, J. Chem. Phys. 134, 114103(2011).\n[15] A. L. Goodman, Nucl. Phys. A 352, 30 (1981).\n[16] J. L. Egido, L. M. Robledo, V. Martin, Phys. Rev. Lett\n85, 26 (2000).\n[17] J. C. Pei, W. Nazarewicz, J. A. Sheikh, and A. K. Ker-\nman, Phys. Rev. Lett 102, 192501 (2009).\n[18] Y. Zhu and J.C. Pei, Phys. Rev. C 90, 054316(2014).\n[19] J. C. Pei, W. Nazarewicz, J. A. Sheikh, and A. K. Ker-\nman, Nucl. Phys. A 834, 381c (2010).\n[20] J. O. Newton, D. G. Popescu, and J. R. Leigh, Phys.\nRev. C 42, 1772(1990).\n[21] M. G. Itkis, Yu. Ts. Oganessian, and V. I. Zagrebaev,\nPhys. Rev. C 65, 044602(2002).\n[22] C.J. Xia, B.X. Sun, E.G. Zhao, S.G. Zhou, Sci. China\nPhys. Mech. Astron. 

In [22]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt  | llm,
         "context": itemgetter("context"),
      }
)

In [24]:
answer = retrieval_augmented_qa_chain.invoke({"question" : "What is Nuclear Fission ?"})

In [26]:
answer["response"].content

'Nuclear fission is a process in which the nucleus of an atom is split into two or more smaller nuclei, along with the release of a large amount of energy. This process can occur spontaneously or can be induced by bombarding the nucleus with a neutron. Nuclear fission is the basis for nuclear power plants and nuclear weapons.'